In [ ]:
#install thermo unit in libraries
!pip install thermo

Calculate pressure drop of pipe system consist of 2 pipe segment. You are given the following information:

Fluid: Water
Pressure: 2 bar
Temperature: 15°C

Flow Rate: 1500 L/min
Pipe Segments:
Pipe 1: Diameter = 100 mm
Pipe 2: Diameter = 125 mm
Lengths: 34 m and 67 m (including a bend)
Elevation Difference: 22 m
Bend Parameters: 90-degree rounded bends with a bend diameter ratio (r_d) of 1.5

In [11]:
from math import *
from fluids.units import *
from thermo.units import Chemical #import function and libraries

#define fluid properties
water = Chemical('water', P=2*u.bar, T=15*u.degC)
rho = water.rho
mu = water.mu

#specify parameters
Q = 1500*u.L/u.min
r_d = 1.5
_, D1, _, _ = nearest_pipe(Di=100*u.mm)
_, D2, _, _ = nearest_pipe(Di=125*u.mm)
L1 = 34*u.m
L2 = (22+45)*u.m
dH = 22*u.m
beta = D1/D2

#calculate reynolds number
V1 = Q/(pi/4*D1**2)
V2 = Q/(pi/4*D2**2)
Re1 = Reynolds(rho=rho, mu=mu, V=V1, D=D1)
Re2 = Reynolds(rho=rho, mu=mu, V=V2, D=D2)

#calculate friction factors
fd1 = friction_factor(Re=Re1, eD=0.0018*u.inch/D1)
fd2 = friction_factor(Re=Re2, eD=0.0018*u.inch/D2)
fd = (fd1+fd2)/2

#calculate pressure drop components
dP = rho*u.gravity*dH

K_D1 = bend_rounded(Di=D1, angle=90*u.degrees, fd=fd, bend_diameters=r_d)
K_D1 += diffuser_conical(D1, D2, angle=30*u.degrees, fd=fd)
K_D1 += K_from_f(fd=fd1, L=L1, D=D1)

K_D2 = bend_rounded(Di=D2, angle=90*u.degrees, fd=fd, bend_diameters=r_d)
K_D2 += K_from_f(fd=fd2, L=L2, D=D2)

#sum up pressure drop
dP += dP_from_K(K=K_D1, rho=rho, V=V1)
dP += dP_from_K(K=K_D2, rho=rho, V=V2)
dP.to(u.Pa)

262808.2444679967 <Unit('pascal')>